*Synthetic data* can help us to evaluate the properties of our learning algorithms and to confirm that our implementations work as expected.

First, we define the *SyntethicRegressionData* class, its input tensor $X$ and output $y = Xw + b + noise$.
Using this class, we generate our data with arbitrary $w$ and $b$ values, we store our data as a tensor using *TensorDataset*, and load our dataset into the *DataLoader* -> an iterable that abstracts the complexity of handling minibatches, reshuffling the data at every epoch, etc.

In [31]:
import random
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

class SyntheticRegressionData(nn.Module):
    def __init__(self, w, b, noise=0.01, num_train=1000, num_val=1000, batch_size=32):
        super().__init__()
        n = num_train + num_val
        self.X = torch.randn(n, len(w))
        noise = torch.randn(n, 1) * noise
        self.y = torch.matmul(self.X, w.reshape((-1,1))) + b + noise
        self.batch_size = batch_size
        #w.reshape((-1,1)) changes w to a column vector with shape [n,1]
        

data = SyntheticRegressionData(w=torch.tensor([2, -3.4]), b=4.2)
dataset = TensorDataset(data.X, data.y)
dataloader = DataLoader(dataset, batch_size=data.batch_size, shuffle=True)

Then, we create a *LinearRegression* class. We fill our weights with values samples from a normal distribution an our bias with zeros. *nn.LazyLinear* is fully connected layer with 1 output feature. It automatically infers __in_features__ during the first pass. When we call forward, it applies a linear transformation. We set our loss function as *nn.MSELoss()* and set *torch.optim.SGD* as our optimizer.

In [34]:
class LinearRegression(nn.Module):
    def __init__(self, lr):
        super().__init__()
        self.net = nn.LazyLinear(1)
        self.net.weight.data.normal_(0, 0.01)
        self.net.bias.data.fill_(0)
    def forward(self, X):
        return self.net(X)

    def loss(self, y_hat, y):
        fn = nn.MSELoss()
        return fn(y_hat, y)
    
    def configure_optimizers(self, lr):
        return torch.optim.SGD(self.parameters(), lr)

model = LinearRegression(lr=0.01)

/home/lucas/stuff/miniconda3/envs/d2ls/lib/python3.9/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [39]:
import torch

class Trainer:
    def __init__(self, num_epochs=10, lr=0.01):
        self.num_epochs = num_epochs  # Store the number of epochs
        self.lr = 0.01

    def fit(self, model, dataloader):
        """Train the model using the provided DataLoader."""
        self.model = model  # Store model inside Trainer
        self.optimizer = self.model.configure_optimizers(lr=self.lr)  # Initialize optimizer
        self.epoch = 0  # Initialize epoch counter

        for self.epoch in range(self.num_epochs):  # Train for num_epochs
            self.fit_epoch(dataloader)

    def fit_epoch(self, dataloader):
        """Perform one training epoch."""
        self.model.train()  # Set model to training mode
        
        for batch_idx, (X_batch, y_batch) in enumerate(dataloader):
            self.optimizer.zero_grad()  # Reset gradients
            y_pred = self.model(X_batch)  # Forward pass
            loss = self.model.loss(y_pred, y_batch)  # Compute loss
            loss.backward()  # Backpropagation
            self.optimizer.step()  # Update model parameters
            
            print(f"Epoch {self.epoch+1}/{self.num_epochs}, Batch {batch_idx+1}, Loss: {loss.item():.4f}")

trainer = Trainer(num_epochs=10)
trainer.fit(model, dataloader)


TypeError: configure_optimizers() missing 1 required positional argument: 'lr'